In [38]:
# import libraries
import os
import speech_recognition as sr
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.utilities import OpenWeatherMapAPIWrapper,WikipediaAPIWrapper,DuckDuckGoSearchAPIWrapper
from langchain.agents import Tool
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.documents import Document
from langchain.llms import OpenAI
import pyttsx3 
from langchain.agents import initialize_agent
from langsmith import utils

In [ ]:
# read .env file
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())



In [ ]:
# required imports for Langsmith tracing
OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="Travel Assistant"



In [ ]:
# cheack the avalibilty for Langsmith tracing
utils.tracing_is_enabled()

True

In [34]:
# assign APIs libraries into variable
weather_api = OpenWeatherMapAPIWrapper()  #https://home.openweathermap.org/api_keys
duck_api =DuckDuckGoSearchAPIWrapper()
wiki_api = WikipediaAPIWrapper()
recognizer = sr.Recognizer()


In [33]:
# configure embeddings model for document processing
embed = OpenAIEmbeddings(model='text-embedding-ada-002') 

# splits documents into chunks for processing
splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)
    

In [32]:
# Wikipidia retriever system
def retriever_tool(split_docs,query):
    
    # creates a FAISS vector store for similarity search
    vectorstore = FAISS.from_documents(split_docs, embed)
    retriever = vectorstore.as_retriever()
    results = retriever.get_relevant_documents(query)
    return "\n".join([doc.page_content for doc in results])


In [ ]:
# Wikipidia search system 
def wiki_search(query):
    data = wiki_api.load(query)
    split_docs = splitter.split_documents(data)
 
    return retriever_tool(split_docs, query)

In [ ]:
# document identification function
def docs(article):
  return Document(
      page_content=article['snippet'],
      metadata={"title": article['title']}
  )

In [ ]:
# DuckDuckGo search system 
def ddg_search(query):
    data = duck_api.results(query,10)
    documents = []
    for raw in data:
     documents.append(docs(raw))
    split_docs = splitter.split_documents(documents)
 
    return retriever_tool(split_docs, query)

In [26]:
# AI model configuration
llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo-instruct")  # or "gpt-4"


In [25]:
# define the custom prompt template for SmartTourGuideAgent
tour_guide_prompt = PromptTemplate(
    input_variables=["query"],
    template="""
    You are a SmartTourGuideAgent. Your task is to help users with their travel-related questions.
    If the user asks about a city or country, provide them with relevant details about it.
    If the user asks for a trip plan, create a basic itinerary for them.

    Answer questions with relevant emojis to make the answers more engaging and fun. For example:
    - For cities, mention landmarks, weather, or activities with emojis.
    - For countries, mention famous attractions or cultural highlights with emojis.
    - For trip plans, create a fun schedule with emojis representing the activities.

    Query: {query}
    Answer:
    """
)

In [24]:
# tools definitions
tools = [
    Tool(
    name="TravelInfoRetriever",
    func=wiki_search,
    description=(
    "You are a SmartTourGuideAgent."
    "If the user asks about a city or country, provide them with details about it."
    "If the user asks for a trip plan, create a basic itinerary for them."
    "- For cities, mention landmarks, weather, or activities with emojis."
    "- For countries, mention famous attractions or cultural highlights with emojis."
    "- For trip plans, create a fun schedule with emojis representing the activities."
    )
    ),

Tool(
    name="Search",
    func=ddg_search,
    description=(
        "Use this tool to search the web for up-to-date information, news, events, travel rules, or details not covered by other tools. "
        "Best for questions like: 'Is the Venice Carnival this year?' or 'Entry rules for Japan 2025.'"
    )
),

  Tool(
        name='Weather',
        func= weather_api.run,
        description=(
        "Use this tool to find **current or forecasted weather information** about a country, city, or travel destination. "
        "Ideal for questions like: 'What's the weather in Rome?', 'Is it rainy in Tokyo?', or 'How cold is it in Iceland in December?'. "
        "Only use this tool when the user is asking specifically about **weather** conditions. "
        "Do not use it for general travel info or sightseeing.")
  )
]

In [23]:
# audio processing functions
def listen_to_user(timeout=10):
    """Convert speech to text with error handling"""
    try:
        with sr.Microphone() as source:
            print("\n🎤 Listening... (Speak now)")
            audio = recognizer.listen(source, timeout=timeout)
        return recognizer.recognize_google(audio)
    except sr.WaitTimeoutError:
        print("⌛ No speech detected, please try again!")
        return ""
    except Exception as e:
        print(f"🔇 : {str(e)}. Try speaking again.")
        return ""

def text_to_speech(text):
    """Convert text to speech"""
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)  # Speed of speech
    engine.setProperty('volume', 1)  # Volume level (0.0 to 1.0)
    engine.say(text)
    engine.runAndWait()

# conversation management
def get_input_method():
    """Get user's preferred input method"""
    while True:
        method = input("\nChoose input method [text/audio]: ").lower()
        if method in ["text", "t"]:
            return input("✍️ Your travel question: ")
        elif method in ["audio", "a"]:
            return listen_to_user()
        print("⚠️ Please enter 'text' or 'audio'")


In [39]:
# initialize conversation memory 
memory = ConversationBufferMemory(memory_key="chat_history")

# agent Initialization
conversational_agent = initialize_agent(
    agent="conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=6,
    agent_kwargs={
        "prompt": tour_guide_prompt
    },
     memory=memory  
)

In [ ]:
# start the interaction  

while True:
    user_input = get_input_method()
    if user_input.lower() in ["goodbye", "stop", "bye"]:
        print("🤖 Agent: Goodbye! 👋")
        break
    conversational_agent(user_input)